In [1]:
import findspark
findspark.init("/home/gagandeep/spark/spark-2.4.5-bin-hadoop2.7")
from pyspark.sql import SparkSession
import os

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
pathToFile = "/media/gagandeep/2E92405C92402AA3/Work/Codes/PythonCodes/SparkLesson/DataFiles/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv"
data = spark.read.csv(pathToFile, inferSchema=True, header=True)

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [5]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train, test = data.randomSplit([0.8,0.2])

In [8]:
als = ALS(maxIter=10, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [9]:
model = als.fit(train)

In [10]:
pred = model.transform(test)

In [11]:
pred.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|     5| 0.97344506|
|     31|   1.0|    19|  1.9021467|
|     31|   1.0|    18| 0.80515504|
|     85|   1.0|    26|  1.8876135|
|     85|   2.0|    20|   1.265231|
|     85|   1.0|     5| 0.91215223|
|     85|   1.0|    23|  0.7343028|
|     85|   3.0|    21|  4.8433037|
|     65|   1.0|    28|  2.0757418|
|     65|   1.0|    16|  0.9530109|
|     53|   3.0|    20|-0.11286947|
|     53|   5.0|     8|  2.2847855|
|     53|   1.0|    23|  0.7936096|
|     53|   1.0|    25| -2.5344718|
|     53|   3.0|    14|  1.5848817|
|     78|   1.0|    27|  0.8540673|
|     78|   1.0|    12|  1.2646258|
|     78|   1.0|     1|  1.0001954|
|     34|   1.0|    15|  2.6444602|
|     81|   5.0|    28|  1.6275266|
+-------+------+------+-----------+
only showing top 20 rows



In [12]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [13]:
rmse = evaluator.evaluate(pred)
rmse

1.9483203454007345

In [14]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

In [15]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      0|    11|
|      6|    11|
|     18|    11|
|     21|    11|
|     22|    11|
|     23|    11|
|     30|    11|
|     50|    11|
|     51|    11|
|     62|    11|
|     67|    11|
|     79|    11|
|     86|    11|
+-------+------+



In [16]:
recommend = model.transform(single_user)
recommend.orderBy('prediction', ascending=False).show()

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|     62|    11|  4.879845|
|     21|    11| 3.3483655|
|     18|    11| 3.0584986|
|     23|    11| 2.3428204|
|     86|    11| 2.2802045|
|     51|    11|  2.150453|
|     22|    11| 1.9456537|
|     67|    11| 1.7587798|
|     50|    11| 1.1116447|
|      0|    11| 1.0227671|
|     30|    11|0.45696056|
|     79|    11|0.12279113|
|      6|    11|-1.0242363|
+-------+------+----------+

